# Steam Data Cleaning
In this part of the project I will be cleaning the dataset for better visualizations on the data.
Data pre-processing is performed using the pandas library --> and then it is again changed into sql database and deployed on heroku using docker images.

### Import the necessary libraries 

In [1]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval

In [2]:
# open JSON
with open("Dataset/database.json") as f:
    data = json.load(f)
    
# convert the JSON to a dataframe
df = pd.DataFrame(data)
df1 = df.T # Take the transpose of the dataframe.
df1.head()

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,total_reviews,controller_support,dlc,demos,recommendations,achievements,reviews,ext_user_account_notice,metacritic,drm_notice
1198490,game,All That Remains: Part 1,1198490,0,False,“Duncan Price is paranoid” they used to say. H...,“Duncan Price is paranoid” they used to say. H...,“Duncan Price is paranoid” they used to say. H...,"English<strong>*</strong>, French, Italian, Ge...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198510,game,Paperball,1198510,0,False,It's time to get rolling!<br><br>Navigate tric...,It's time to get rolling!<br><br>Navigate tric...,It's time to get rolling! Navigate tricky pass...,"English<strong>*</strong>, German, Japanese, D...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,134,full,"[1209620, 1209621, 1285150]","[{'appid': 1219680, 'description': ''}]",{'total': 143},"{'total': 37, 'highlighted': [{'name': 'Ready ...",NaN,NaN,NaN,NaN
1198550,game,Moderium,1198550,0,False,Moderium is a Wave-Based Isometric Action Role...,Moderium is a Wave-Based Isometric Action Role...,Conquer waves of encroaching enemies in this A...,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,1,NaN,NaN,"[{'appid': 1220740, 'description': ''}]",NaN,NaN,NaN,NaN,NaN,NaN
1198600,game,Pizza Time Explosion,1198600,0,False,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",Welcome to the Pizza Dimension!,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,18,full,[1220660],NaN,NaN,"{'total': 32, 'highlighted': [{'name': 'Pizza ...",NaN,NaN,NaN,NaN
1198630,game,Customer Cums First!,1198630,0,False,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",It's just like any other shopping district in ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,10,NaN,NaN,NaN,NaN,"{'total': 18, 'highlighted': [{'name': 'Now th...",NaN,NaN,NaN,NaN


In [3]:
# Read it into a csv file.
df1.to_csv('steam.csv')

In [2]:
steam = pd.read_csv("steam.csv")

In [3]:
# To check the number of null values in each column. 
null_count = steam.isna().sum()
null_count

Unnamed: 0                    0
type                          0
name                          0
steam_appid                   0
required_age                  0
is_free                       0
detailed_description          1
about_the_game                1
short_description             2
supported_languages           0
header_image                  0
website                    1673
pc_requirements               0
mac_requirements              0
linux_requirements            0
legal_notice               3056
developers                    3
publishers                    0
price_overview              372
packages                    336
package_groups                0
platforms                     0
categories                   69
genres                        3
screenshots                   2
movies                       79
release_date                  0
support_info                  0
background                    2
content_descriptors           0
num_reviews                   0
review_s

## Data Pre-Processing
### Drop columns with null values.

These columns contain more than 50% null values so we will drop them --> `'controller_support', 'dlc', 'demos', 'recommendations', 'reviews', 'ext_user_account_notice', 'metacritic', 'drm_notice'` and `'legal_notice'`

The first function will remove the columns with more than 50% missing values, taking care of the columns with high null counts. We can do this by running a filter on the dataframe, as seen below.

In [4]:
threshold = steam.shape[0] // 2

drop_col = steam.columns[null_count > threshold]

print('Columns to drop: {}'.format(list(drop_col)))


Columns to drop: ['legal_notice', 'controller_support', 'dlc', 'demos', 'recommendations', 'reviews', 'ext_user_account_notice', 'metacritic', 'drm_notice']


In [5]:
steam.drop(columns=drop_col, inplace=True)

In [6]:
#Also drop the Unnamed: 0 column which is the appid
steam.drop(columns = ['Unnamed: 0'], inplace=True)

### Delete the duplicate rows if they have same `steam_appid`

In [7]:
# Delete the duplicate rows in the dataset where each game has a unique steam_appid.
duplicate_rows = steam[steam.duplicated(subset='steam_appid')] 

print('Duplicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head()

Duplicate rows to remove: 3


,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,support_info,background,content_descriptors,num_reviews,review_score,review_score_desc,total_positive,total_negative,total_reviews,achievements
1864,game,Tom Clancy's Rainbow Six® Siege,359550,0,False,"<h1>Edition Comparison</h1><p><img src=""https:...",Master the art of destruction and gadgetry in ...,Tom Clancy's Rainbow Six Siege is the latest i...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"{'url': 'http://support.ubi.com', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",2,0,2 user reviews,1,1,2,NaN
1865,game,Tom Clancy's Rainbow Six® Siege,359550,0,False,"<h1>Edition Comparison</h1><p><img src=""https:...",Master the art of destruction and gadgetry in ...,Tom Clancy's Rainbow Six Siege is the latest i...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"{'url': 'http://support.ubi.com', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",2,0,2 user reviews,2,0,2,NaN
1866,game,Tom Clancy's Rainbow Six® Siege,359550,0,False,"<h1>Edition Comparison</h1><p><img src=""https:...",Master the art of destruction and gadgetry in ...,Tom Clancy's Rainbow Six Siege is the latest i...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"{'url': 'http://support.ubi.com', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",2,0,2 user reviews,1,1,2,NaN


In [8]:
steam.drop_duplicates(subset ="steam_appid",keep = 'first', inplace = True)

In [9]:
steam.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'header_image', 'website', 'pc_requirements',
       'mac_requirements', 'linux_requirements', 'developers', 'publishers',
       'price_overview', 'packages', 'package_groups', 'platforms',
       'categories', 'genres', 'screenshots', 'movies', 'release_date',
       'support_info', 'background', 'content_descriptors', 'num_reviews',
       'review_score', 'review_score_desc', 'total_positive', 'total_negative',
       'total_reviews', 'achievements'],
      dtype='object')

### Processing the `price_overview` column to extract the price of each game.

The column `is_free` and `price_overview` are linked with respect to the price of each game. 
If `is_free` is true and `price_overview` has a null value then it means that the price of that game is €0.0

In [10]:
# check how many null values are in price_overview
steam['price_overview'].isnull().sum()

372

In [11]:
price_zero = steam[(steam['is_free']) & (steam['price_overview'].isnull())]
len(price_zero)

276

This shows that there are 276 games have a price of €0.0

Let's set the final price value for these columns as 0.0

Also there are 96 games whose price value is not stated and it states that these games are not free.


In [12]:
steam['price_overview'][0]

"{'currency': 'EUR', 'initial': 239, 'final': 239, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '2,39€'}"

##### Lets create separate columns `Final_price` and `Discount` and extract from `price_overview`.
We know that the currency of the price is in EURO.

In [13]:
def process_price(steam):
    steam = steam.copy()
        
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'discount_percent':0, 'initial': -1,'currency':'EUR'}
    
    # evaluate as dictionary and set to -1 if missing
    steam['price_overview'] = steam['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    steam['Discount'] = steam['price_overview'].apply(lambda x: x['discount_percent'])
    steam['Price'] = steam['price_overview'].apply(lambda x: x['initial'])
    steam['Currency'] = steam['price_overview'].apply(lambda x: x['currency'])
    
    # Set price of free games to 0
    steam.loc[steam['is_free'], 'Price'] = 0
    
    return steam

price_data = process_price(steam)[['name', 'Discount', 'Price','Currency']]
price_data.head()

,name,Discount,Price,Currency
0,All That Remains: Part 1,0,239,EUR
1,Paperball,0,999,EUR
2,Moderium,0,79,EUR
3,Pizza Time Explosion,0,499,EUR
4,Customer Cums First!,0,999,EUR


In [14]:
price_data[price_data['Currency'] != 'EUR']

,name,Discount,Price,Currency
95,Mokoko,0,499,USD
202,DEATH STRANDING,0,7999,CAD
428,DeadShotZ,0,199,USD
914,PANDARA,0,400,CHF
937,Sakura Fox Adventure,0,999,USD
948,Three Of a Fish,0,99,USD
1023,Night Road,0,289,GBP
1096,Prison Princess,0,1549,GBP
1730,Battle Knights,0,299,USD
1843,The Elder Scrolls V: Skyrim Special Edition,60,4616000,KRW


In [18]:
def process_price(steam):
    """Process price_overview column into formatted price column."""
    steam = steam.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'discount_percent':0, 'currency': 'EUR', 'initial': -1}
    
    # evaluate as dictionary and set to -1 if missing
    steam['price_overview'] = steam['price_overview'].apply(parse_price)
    
    # create columns from currency and initial values
    steam['discount'] = steam['price_overview'].apply(lambda x: x['discount_percent'])
    steam['price'] = steam['price_overview'].apply(lambda x: x['initial'])
    steam['currency'] = steam['price_overview'].apply(lambda x: x['currency'])
    
    # set price of free games to 0
    steam.loc[steam['is_free'], 'price'] = 0
    
    # remove non-EUR rows
    steam = steam[steam['currency'] == 'EUR']
    
    # change price to display in pounds (only applying to rows with a value greater than 0)
    steam.loc[steam['price'] > 0, 'price'] /= 100
    
    # remove columns no longer needed
    steam = steam.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    return steam


price_steam = process_price(steam)
price_steam[['name', 'price']]

,name,price
0,All That Remains: Part 1,2.39
1,Paperball,9.99
2,Moderium,0.79
3,Pizza Time Explosion,4.99
4,Customer Cums First!,9.99
...,...,...
4620,Lunch A Palooza,10.79
4621,Uncharted Tides: Port Royal,14.99
4622,Persona 4 Golden,19.99
4623,Magic Farm 3: The Ice Danger,16.79


There are 15 games with different currency. Let's convert them to EURO with today's conversion rate and get the price.

In [60]:
"""def price(price_data):
    price_data.loc[price_data['Currency'] == 'USD', 'Currency'] = 'EUR'
    price_data.loc[price_data['Currency']=='USD', 'Price'] *= 0.89
        

dfprice = price(price_data)
dfprice   """

### Processing the `genres` column to extract the GENRE of each game.
Let's have a look at one of the value of genre column.

In [19]:
# Let's have a look at one of the genres value
steam['genres'][50]

"[{'id': '25', 'description': 'Adventure'}, {'id': '23', 'description': 'Indie'}]"

In [20]:
print('Categories:\n')
print('Null values:', steam['categories'].isnull().sum())
print()
print(steam['categories'][0])

print('\nGenres:\n')
print('Null values:', steam['genres'].isnull().sum())
print()
print(steam['genres'].iloc[0])
print(steam['genres'].iloc[1000])


Categories:

Null values: 69

[{'id': 2, 'description': 'Single-player'}, {'id': 23, 'description': 'Steam Cloud'}]

Genres:

Null values: 3

[{'id': '25', 'description': 'Adventure'}]
[{'id': '4', 'description': 'Casual'}, {'id': '23', 'description': 'Indie'}, {'id': '28', 'description': 'Simulation'}]


In [21]:
def print_steam_links(df):
    """Print links to store page for apps in a dataframe."""
    url_base = "https://store.steampowered.com/app/"
    
    for i, row in df.iterrows():
        appid = row['steam_appid']
        name = row['name']
        
        print(name + ':', url_base + str(appid))


In [22]:
print_steam_links(steam[steam['categories'].isnull()].sample(5, random_state=0))

CyberLink PowerDirector 18 Ultra - Video editing, Video editor, making videos: https://store.steampowered.com/app/1127870
Web Designer 12 Premium Steam Edition: https://store.steampowered.com/app/485150
Marmoset Hexels 3: https://store.steampowered.com/app/428340
Discord Bot Studio: https://store.steampowered.com/app/1118380
MindTex 2: https://store.steampowered.com/app/441770


In [23]:
steam.shape

(4622, 35)

In [25]:
price_steam = price_steam.dropna(subset=['categories'])
price_steam.shape

(4539, 35)

In [28]:
price_steam = price_steam.dropna(subset=['genres'])
price_steam.shape

(4537, 35)

In [29]:
def process_categories_and_genres(steam):
    steam = steam.copy()
    steam = steam[(steam['categories'].notnull()) & (steam['genres'].notnull())]
    
    for col in ['categories', 'genres']:
        steam[col] = steam[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))
    
    return steam

cat_gen_steam = process_categories_and_genres(price_steam)
cat_gen_steam[['steam_appid', 'categories', 'genres']].head()

,steam_appid,categories,genres
0,1198490,Single-player;Steam Cloud,Adventure
1,1198510,Single-player;Multi-player;PvP;Shared/Split Sc...,Action;Indie
2,1198550,Single-player,Action;Casual;Indie;RPG
3,1198600,Single-player;Steam Achievements;Full controll...,Action;Indie
4,1198630,Single-player;Steam Achievements,Casual


In [30]:
cat_gen_steam.shape

(4537, 35)

### Processing `developers` and `publishers` column.

In [33]:
print('Developers null counts:', cat_gen_steam['developers'].isnull().sum())
print('Developers empty list counts:', cat_gen_steam[cat_gen_steam['developers'] == "['']"].shape[0])

print('\nPublishers null counts:', cat_gen_steam['publishers'].isnull().sum())
print('Publishers empty list counts:', cat_gen_steam[cat_gen_steam['publishers'] == "['']"].shape[0])

Developers null counts: 3
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 31


In [32]:
steam['developers'][50]

"['Volrest']"

In [125]:
steam[['developers', 'publishers']].iloc[24:28]

,developers,publishers
24,['Turnfollow'],['Tender Claws']
25,['Flight School Studio'],['MWM Interactive']
26,['Elephant Games'],['Big Fish Games']
27,['Domini Games'],['Big Fish Games']


In [34]:
def process_developers_and_publishers(steam):
    # remove rows with missing data
    steam = steam[(steam['developers'].notnull()) & (steam['publishers'] != "['']")].copy()
    
    for col in ['developers', 'publishers']:
        steam[col] = steam[col].apply(lambda x: literal_eval(x))
        
        # filter dataframe to rows with lists longer than 1, and store the number of rows
        num_rows = steam[steam[col].str.len() > 1].shape[0]
        
        print('Rows in {} column with multiple values:'.format(col), num_rows)

process_developers_and_publishers(cat_gen_steam)

Rows in developers column with multiple values: 296
Rows in publishers column with multiple values: 148


In [128]:
steam[(steam['publishers'] == "['NA']") | (steam['developers'] == "['N/A']")].shape[0]

3

In [35]:
def process_developers_and_publishers(steam):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    steam = steam[(steam['developers'].notnull()) & (steam['publishers'] != "['']")].copy()
    steam = steam[~(steam['developers'].str.contains(';')) & ~(steam['publishers'].str.contains(';'))]
    steam = steam[(steam['publishers'] != "['NA']") & (steam['publishers'] != "['N/A']")]
    
    # create list for each
    steam['developer'] = steam['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    steam['publisher'] = steam['publishers'].apply(lambda x: ';'.join(literal_eval(x)))

    steam = steam.drop(['developers', 'publishers'], axis=1)
    
    return steam

dev_pub_steam = process_developers_and_publishers(cat_gen_steam)
dev_pub_steam[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
0,All That Remains: Part 1,1198490,Glitch Games,Glitch Games
1,Paperball,1198510,Cliax Games,Cliax Games
2,Moderium,1198550,All Saints Gaming,All Saints Gaming
3,Pizza Time Explosion,1198600,Electric Prune Juice,Electric Prune Juice
4,Customer Cums First!,1198630,Miel,Cherry Kiss Games


In [36]:
dev_pub_steam.shape

(4500, 35)

In [38]:
def process_achievements(steam):
    """Parse as total number of achievements."""
    steam = steam.copy()
    
    steam = steam.drop('content_descriptors', axis=1)
    
    def parse_achievements(x):
        if x is np.nan:
            # missing data, assume has no achievements
            return 0
        else:
            # else has data, so can extract and return number under total
            return literal_eval(x)['total']
        
    steam['achievements'] = steam['achievements'].apply(parse_achievements)
    
    return steam

achiev_steam = process_achievements(dev_pub_steam)
achiev_steam['achievements'].value_counts().head()

0     1713
10     139
12     102
8       93
13      87
Name: achievements, dtype: int64

### Processing the `platforms` column.

In [39]:
steam['platforms'].value_counts()

{'windows': True, 'mac': False, 'linux': False}    3223
{'windows': True, 'mac': True, 'linux': True}       694
{'windows': True, 'mac': True, 'linux': False}      562
{'windows': True, 'mac': False, 'linux': True}      143
Name: platforms, dtype: int64

In [40]:
steam['platforms'].isnull().sum()

0

In [45]:
def process_platforms(steam):
    """Split platforms column into separate boolean columns for each platform."""
    # evaluate values in platforms column, so can index into dictionaries
    steam = steam.copy()
    
    def parse_platforms(x):
        
        d = literal_eval(x)
        
        return ';'.join(platform for platform in d.keys() if d[platform])
    
    steam['platforms'] = steam['platforms'].apply(parse_platforms)
    
    return steam


steam1 = process_platforms(achiev_steam)
steam1['platforms'].value_counts()

windows              3131
windows;mac;linux     679
windows;mac           550
windows;linux         140
Name: platforms, dtype: int64

In [46]:
steam1.shape

(4500, 34)

In [47]:
steam1.isnull().sum()

type                       0
name                       0
steam_appid                0
required_age               0
detailed_description       1
about_the_game             1
short_description          2
supported_languages        0
header_image               0
website                 1638
pc_requirements            0
mac_requirements           0
linux_requirements         0
packages                 315
package_groups             0
platforms                  0
categories                 0
genres                     0
screenshots                2
movies                    76
release_date               0
support_info               0
background                 2
num_reviews                0
review_score               0
review_score_desc          0
total_positive             0
total_negative             0
total_reviews              0
achievements               0
discount                   0
price                      0
developer                  0
publisher                  0
dtype: int64

In [48]:
steam1[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

detailed_description    1
about_the_game          1
short_description       2
dtype: int64

In [53]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.
    
    filename : str without file extension
    """
    filepath = 'steam_' + filename + '.csv'
    
    df.to_csv(filepath, index=False)
    
    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))

In [54]:
def process_descriptions(steam, export=False):
    """Export descriptions to external csv file then remove these columns."""
    # remove rows with missing description data
    steam = steam[steam['detailed_description'].notnull()].copy()
    
    # remove rows with unusually small description
    steam = steam[steam['detailed_description'].str.len() > 20]
    
    # by default we don't export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = steam[['steam_appid', 'detailed_description', 'about_the_game', 'short_description']]
        
        export_data(description_data, filename='description_data')
    
    # drop description columns from main dataframe
    steam = steam.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)
    
    return steam

desc_steam = process_descriptions(steam1, export=True)

Exported description data to 'steam_description_data.csv'


In [55]:
steam1.shape

(4500, 34)

In [57]:
image_cols = ['header_image', 'screenshots', 'background']

for col in image_cols:
    print(col+':', desc_steam[col].isnull().sum())

desc_steam[image_cols].head()

header_image: 0
screenshots: 1
background: 1


,header_image,screenshots,background
0,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
1,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
2,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
3,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
4,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...


In [58]:
def process_media(steam, export=False):
    """Remove media columns from dataframe, optionally exporting them to csv first."""
    steam = steam[steam['screenshots'].notnull()].copy()
    
    if export:
        media_data = steam[['steam_appid', 'header_image', 'screenshots', 'background', 'movies']]
        
        export_data(media_data, 'media_data')
        
    steam = steam.drop(['header_image', 'screenshots', 'background', 'movies'], axis=1)
    
    return steam

media_steam = process_media(desc_steam, export=True)

Exported media data to 'steam_media_data.csv'


In [60]:
print('website null counts:', media_steam['website'].isnull().sum())
print('support_info null counts:', media_steam['support_info'].isnull().sum())

with pd.option_context("display.max_colwidth", 100): # ensures strings not cut short
    display(media_steam[['name', 'website', 'support_info']][75:80])

website null counts: 1637
support_info null counts: 0


,name,website,support_info
75,The Pale City,https://thepalecity.com/,"{'url': 'https://thepalecity.com/contact/', 'email': ''}"
76,Nexomon: Extinction,http://www.nexomongame.com,"{'url': 'https://www.facebook.com/NexomonGame/', 'email': 'support@vewointeractive.com'}"
77,OctaFight,http://octafight.com/,"{'url': '', 'email': 'support@octafight.com'}"
78,Dead Dreams,NaN,"{'url': '', 'email': 'aiazmarx@gmail.com'}"
79,Solar Panic: Utter Distress,http://www.barracudadisaster.com/solarpanic/,"{'url': '', 'email': 'contact@barracudadisaster.com'}"


In [61]:
def process_info(steam, export=False):
    """Drop support information from dataframe, optionally exporting beforehand."""
    if export:
        support_info = steam[['steam_appid', 'website', 'support_info']].copy()
        
        support_info['support_info'] = support_info['support_info'].apply(lambda x: literal_eval(x))
        support_info['support_url'] = support_info['support_info'].apply(lambda x: x['url'])
        support_info['support_email'] = support_info['support_info'].apply(lambda x: x['email'])
        
        support_info = support_info.drop('support_info', axis=1)
        
        # only keep rows with at least one piece of information
        support_info = support_info[(support_info['website'].notnull()) | (support_info['support_url'] != '') | (support_info['support_email'] != '')]

        export_data(support_info, 'support_info')
    
    steam = steam.drop(['website', 'support_info'], axis=1)
    
    return steam


info_steam = process_info(media_steam, export=True)

Exported support info to 'steam_support_info.csv'


In [64]:
def process(steam):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    steam = steam.copy()
    steam = process_descriptions(steam, export=True)
    steam = process_media(steam, export=True)
    steam = process_info(steam, export=True)
    #steam = process_requirements(steam, export=True)
    return steam

steam_data = process(steam1)
steam_data.head()


Exported description data to 'steam_description_data.csv'
Exported media data to 'steam_media_data.csv'
Exported support info to 'steam_support_info.csv'


,type,name,steam_appid,required_age,supported_languages,pc_requirements,mac_requirements,linux_requirements,packages,package_groups,...,review_score,review_score_desc,total_positive,total_negative,total_reviews,achievements,discount,price,developer,publisher
0,game,All That Remains: Part 1,1198490,0,"English<strong>*</strong>, French, Italian, Ge...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[413222],"[{'name': 'default', 'title': 'Buy All That Re...",...,0,4 user reviews,1,3,4,0,0,2.39,Glitch Games,Glitch Games
1,game,Paperball,1198510,0,"English<strong>*</strong>, German, Japanese, D...",{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[413229],"[{'name': 'default', 'title': 'Buy Paperball',...",...,8,Very Positive,130,4,134,37,0,9.99,Cliax Games,Cliax Games
2,game,Moderium,1198550,0,English<strong>*</strong><br><strong>*</strong...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[413245],"[{'name': 'default', 'title': 'Buy Moderium', ...",...,0,1 user reviews,1,0,1,0,0,0.79,All Saints Gaming,All Saints Gaming
3,game,Pizza Time Explosion,1198600,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[413271],"[{'name': 'default', 'title': 'Buy Pizza Time ...",...,7,Positive,17,1,18,32,0,4.99,Electric Prune Juice,Electric Prune Juice
4,game,Customer Cums First!,1198630,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[413283],"[{'name': 'default', 'title': 'Buy Customer Cu...",...,7,Positive,8,2,10,18,0,9.99,Miel,Cherry Kiss Games


In [65]:
steam_data.isnull().sum()

type                     0
name                     0
steam_appid              0
required_age             0
supported_languages      0
pc_requirements          0
mac_requirements         0
linux_requirements       0
packages               313
package_groups           0
platforms                0
categories               0
genres                   0
release_date             0
num_reviews              0
review_score             0
review_score_desc        0
total_positive           0
total_negative           0
total_reviews            0
achievements             0
discount                 0
price                    0
developer                0
publisher                0
dtype: int64

In [66]:
steam_data.drop(columns = ['packages'], inplace=True)

In [68]:
steam_data.to_csv('clea_data.csv', index=False)

### Processing Release date

In [70]:
import re
def process_release_date(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x['coming_soon']:
            return '' # return blank string so can drop missing at end
        else:
            return x['date']
    
    df['release_date'] = df['release_date'].apply(eval_date)
    
    def parse_date(x):
        if re.search(r'[\d]{1,2} [A-Za-z]{3}, [\d]{4}', x):
            return x.replace(',', '')
        elif re.search(r'[A-Za-z]{3} [\d]{4}', x):
            return '1 ' + x
        elif x == '':
            return np.nan
        else:
            # Should be everything, print out anything left just in case
            print(x)
            
    df['release_date'] = df['release_date'].apply(parse_date)
    df['release_date'] = pd.to_datetime(df['release_date'], format='%d %b %Y', errors='coerce')
    
    df = df[df['release_date'].notnull()]
    
    return df

steam_new = process_release_date(steam_data)
steam_new.head()

Dec 7, 2020


,type,name,steam_appid,required_age,supported_languages,pc_requirements,mac_requirements,linux_requirements,package_groups,platforms,...,review_score,review_score_desc,total_positive,total_negative,total_reviews,achievements,discount,price,developer,publisher
0,game,All That Remains: Part 1,1198490,0,"English<strong>*</strong>, French, Italian, Ge...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],"[{'name': 'default', 'title': 'Buy All That Re...",windows;mac,...,0,4 user reviews,1,3,4,0,0,2.39,Glitch Games,Glitch Games
1,game,Paperball,1198510,0,"English<strong>*</strong>, German, Japanese, D...",{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Paperball',...",windows,...,8,Very Positive,130,4,134,37,0,9.99,Cliax Games,Cliax Games
2,game,Moderium,1198550,0,English<strong>*</strong><br><strong>*</strong...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Moderium', ...",windows,...,0,1 user reviews,1,0,1,0,0,0.79,All Saints Gaming,All Saints Gaming
3,game,Pizza Time Explosion,1198600,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Pizza Time ...",windows,...,7,Positive,17,1,18,32,0,4.99,Electric Prune Juice,Electric Prune Juice
4,game,Customer Cums First!,1198630,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"[{'name': 'default', 'title': 'Buy Customer Cu...",windows;mac;linux,...,7,Positive,8,2,10,18,0,9.99,Miel,Cherry Kiss Games


In [72]:
steam_new['release_date'][50]

Timestamp('2020-01-30 00:00:00')

In [73]:
steam_new[steam_new['release_date'] > '2019-05-01']

,type,name,steam_appid,required_age,supported_languages,pc_requirements,mac_requirements,linux_requirements,package_groups,platforms,...,review_score,review_score_desc,total_positive,total_negative,total_reviews,achievements,discount,price,developer,publisher
0,game,All That Remains: Part 1,1198490,0,"English<strong>*</strong>, French, Italian, Ge...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],"[{'name': 'default', 'title': 'Buy All That Re...",windows;mac,...,0,4 user reviews,1,3,4,0,0,2.39,Glitch Games,Glitch Games
1,game,Paperball,1198510,0,"English<strong>*</strong>, German, Japanese, D...",{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Paperball',...",windows,...,8,Very Positive,130,4,134,37,0,9.99,Cliax Games,Cliax Games
2,game,Moderium,1198550,0,English<strong>*</strong><br><strong>*</strong...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Moderium', ...",windows,...,0,1 user reviews,1,0,1,0,0,0.79,All Saints Gaming,All Saints Gaming
3,game,Pizza Time Explosion,1198600,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Pizza Time ...",windows,...,7,Positive,17,1,18,32,0,4.99,Electric Prune Juice,Electric Prune Juice
4,game,Customer Cums First!,1198630,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"[{'name': 'default', 'title': 'Buy Customer Cu...",windows;mac;linux,...,7,Positive,8,2,10,18,0,9.99,Miel,Cherry Kiss Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4620,game,Lunch A Palooza,1113770,0,"English, French, Italian, German, Spanish - Sp...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],"[{'name': 'default', 'title': 'Buy Lunch A Pal...",windows;mac,...,7,Positive,12,1,13,21,0,10.79,Seashell Studio,Alternative Software Ltd
4621,game,Uncharted Tides: Port Royal,1113780,0,"English<strong>*</strong>, French, Italian, Ge...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"[{'name': 'default', 'title': 'Buy Uncharted T...",windows;mac;linux,...,7,Positive,26,4,30,22,0,14.99,Cordelia Games,Artifex Mundi
4622,game,Persona 4 Golden,1113000,0,"English<strong>*</strong>, Japanese<strong>*</...",{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Persona 4 G...",windows,...,9,Overwhelmingly Positive,29369,629,29998,50,0,19.99,ATLUS,SEGA
4623,game,Magic Farm 3: The Ice Danger,1113060,0,English,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"[{'name': 'default', 'title': 'Buy Magic Farm ...",windows,...,0,4 user reviews,4,0,4,27,0,16.79,Meridian'93,Meridian'93


In [75]:
steam_new.drop(columns = ['supported_languages','pc_requirements','mac_requirements','linux_requirements'], inplace=True)

In [76]:
steam_new.to_csv('clean_data.csv', index=False)

In [77]:
steam_new.isnull().sum()

type                 0
name                 0
steam_appid          0
required_age         0
package_groups       0
platforms            0
categories           0
genres               0
release_date         0
num_reviews          0
review_score         0
review_score_desc    0
total_positive       0
total_negative       0
total_reviews        0
achievements         0
discount             0
price                0
developer            0
publisher            0
dtype: int64

# Boxlieter method